# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as ols
x = boston_features
y = pd.DataFrame(boston.target, columns=["price"])
x_int = sm.add_constant(x)
model = sm.OLS(y, x).fit()
model.summary()

/Users/joe/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Fri, 15 Feb 2019   Prob (F-statistic):          5.15e-153
Time:                        16:32:51   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
CRIM              -1.9000      2.091     -0.909      0.364      -6.009       2.209
INDUS             -0.8069      0.362     -2.228      0.026      -1.518      -0.095
CHAS               2.5968      0.796      3.262      0.001       1.033       4.161
RM                 2.6445      0.408      6.480      0.000       1.843       3.446
AGE                0.0787      0.352      0.224      0.823      -0.612       0.770
DIS              -10.0839      1.855     -5.437      0.000     -13.728      -6.440
PTRATIO           -1.4864      0.241     -6.159      0.000      -1.961      -1.012
B                  3.8623      0.981      3.935      0.000       1.934       5.791
LSTAT             -5.6315      0.354    -15.929      0.000      -6.326      -4.937
RAD_(0, 6]         4.2201      1.857      2.272      0.024       0.571       7.870
RAD_(6, 24]        5.5473      2.117      2.620      0.009       1.387       9.708
TAX_(0, 270]       4.3925      1.371      3.204      0.001       1.699       7.086
TAX_(270, 360]     3.1312      1.375      2.277      0.023       0.429       5.833
TAX_(360, 712]     2.2438      1.333      1.683      0.093      -0.375       4.863
==============================================================================
Omnibus:                      106.736   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              431.931
Skew:                           0.891   Prob(JB):                     1.61e-94
Kurtosis:                       7.161   Cond. No.                     6.45e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.12e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [20]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(x, y)
linreg.coef_

array([[ -1.89995316,  -0.80688617,   2.59684028,   2.64453176,
          0.0786663 , -10.0839112 ,  -1.48638161,   3.86233002,
         -5.63145746,  -0.66357383,   0.66357383,   1.13664819,
         -0.12462051,  -1.01202768]])

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [22]:
new_x = x.iloc[:,0:9]
new_x.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114


In [24]:
x_int = sm.add_constant(new_x)
model = sm.OLS(y, new_x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     1643.
Date:                Mon, 07 Jan 2019   Prob (F-statistic):               0.00
Time:                        13:58:03   Log-Likelihood:                -1466.3
No. Observations:                 506   AIC:                             2951.
Df Residuals:                     497   BIC:                             2989.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM           0.3747      1.490      0.251      0.802      -2.553       3.303
INDUS         -1.0831      0.348     -3.113      0.002      -1.767      -0.400
CHAS           2.7675      0.801      3.455      0.001       1.194       4.341
RM             3.4871      0.216     16.156      0.000       3.063       3.911
AGE           -0.0626      0.349     -0.179      0.858      -0.748       0.623
DIS           -8.9963      1.632     -5.513      0.000     -12.202      -5.790
PTRATIO       -1.3444      0.230     -5.846      0.000      -1.796      -0.893
B              4.7595      0.877      5.425      0.000       3.036       6.483
LSTAT         -5.3114      0.310    -17.144      0.000      -5.920      -4.703
==============================================================================
Omnibus:                      114.956   Durbin-Watson:                   1.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              531.874
Skew:                           0.921   Prob(JB):                    3.20e-116
Kurtosis:                       7.673   Cond. No.                         57.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Statsmodels

### Scikit-learn

In [25]:
linreg = LinearRegression()
linreg.fit(new_x, y)
linreg.coef_

array([[-0.56881564, -1.17237121,  2.7950424 ,  2.88249973,  0.01066422,
        -9.91282921, -1.34474808,  3.95381068, -5.62902505]])

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.